## Problem 

1. the tfjs algorithm does not work for the matrix with the shape of more than $3\times3$
    (I was trying to debug but the logic is correct, and I was unable to fix it)
2. the inversed matrix will be printed multiple times both for print out to console and append to info list. 
3. for $3\times 3$ matrix, the running time of using numpy is faster.

In [1]:
import jp_proxy_widget

In [2]:
w = jp_proxy_widget.JSProxyWidget()

js_file = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs@2.0.0/dist/tf.min.js"
w.load_js_files([js_file])

In [19]:
import numpy as np

In [108]:
%%time
m1 = np.random.randint(100,size=(3, 3))
np.linalg.inv(m1) 

CPU times: user 169 µs, sys: 13 µs, total: 182 µs
Wall time: 177 µs


array([[-0.1574305 ,  0.22938025, -0.22065207],
       [ 0.01186848, -0.03107971,  0.04479542],
       [ 0.01588621,  0.0012007 , -0.00618823]])

In [109]:
m1

array([[ 3, 25, 74],
       [17, 97, 96],
       [11, 83, 47]])

In [94]:
np.linalg.det(m1) 

-398806.99999999977

In [110]:
m2 = m1.flatten().tolist()
m2_shape = [4,4]

In [111]:
m2

[3, 25, 74, 17, 97, 96, 11, 83, 47]

In [153]:
info = []
def callback(args):
    info.append(args)

In [159]:
%%time
w.element.test_func(m2, m2_shape)

CPU times: user 2.02 ms, sys: 127 µs, total: 2.15 ms
Wall time: 2.08 ms


['method', <class 'jp_proxy_widget.proxy_widget.CommandMaker'>::140616804538912, 'test_func', <class 'jp_proxy_widget.proxy_widget.LiteralMaker'>::140616808086608, <class 'jp_proxy_widget.proxy_widget.LiteralMaker'>::140616808085376]

In [160]:
info

[{},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 {},
 [[-92.26052856445312, -6.731898784637451, 35.656715393066406],
  [18.71006965637207, 1.3821560144424438, -7.238805770874023],
  [-2.567164182662964, -0.1940298080444336, 1]],
 [[-92.26052856445312, -6.731898784637451, 35.656715393066406],
  [18.71006965637207, 1.3821560144424438, -7.238805770874023],
  [-2.567164182662964, -0.1940298080444336, 1]],
 [[-92.26052856445312, -6.731898784637451, 35.656715393066406],
  [18.71006965637207, 1.3821560144424438, -7.238805770874023],
  [-2.567164182662964, -0.1940298080444336, 1]],
 [[-92.26052856445312, -6.731898784637451, 35.656715393066406],
  [18.71006965637207, 1.3821560144424438, -7.238805770874023],
  [-2.567164182662964, -0.1940298080444336, 1]],
 [[-92.26052856445312, -6.731898784637451, 35.656715393066406],
  [18.71006965637207, 1.3821560144424438, -7.238805770874023],
  [-2.567164182662964, -0.1940298080444336, 1]],
 [[-92.26052856445312, -6.731898784637451, 35.65671539306640

In [158]:
w.js_init("""
element.empty()
element.test_func = function(input_matrix, input_shape){
    // calculate the determinant of a matrix m
    function det(m) {
        return tf.tidy(() => {
            const [r, _] = m.shape
            if (r === 2) {
                const t = m.as1D()
                const a = t.slice([0], [1]).dataSync()[0]
                const b = t.slice([1], [1]).dataSync()[0]
                const c = t.slice([2], [1]).dataSync()[0]
                const d = t.slice([3], [1]).dataSync()[0]
                result = a * d - b * c
                return result

            } else {
                let s = 0;
                rows = [...Array(r).keys()]
                for (let i = 0; i < r; i++) {
                    sub_m = m.gather(tf.tensor1d(rows.filter(e => e !== i), 'int32'))
                    sli = sub_m.slice([0, 1], [r - 1, r - 1])
                    s += Math.pow(-1, i) * det(sli)
                }
                return s
            }
        })
    }

    // the inverse of the matrix : jordan-gauss method
    function invertM(m) {
        return tf.tidy(() => {
            if (det(m) === 0) {
                return
            }
            const [r, _] = m.shape
            let inv = m.concat(tf.eye(r), 1)
            rows = [...Array(r).keys()]
            for (let i = 0; i < r; i++) {
                inv = tf.tidy(() => {
                    for (let j = i + 1; j < r; j++) {
                        const elt = inv.slice([j, i], [1, 1]).as1D().asScalar()
                        const pivot = inv.slice([i, i], [1, 1]).as1D().asScalar()
                        let newrow
                        if (elt.dataSync()[0] !== 0) {
                            newrow = inv.gather(tf.tensor1d([i], 'int32')).sub(inv.gather(tf.tensor1d([j], 'int32')).div(elt).mul(pivot)).as1D()
                            const sli = inv.gather(rows.filter(e => e !== j))
                            const arr = []
                            if (j === 0) {
                                arr.push(newrow)
                            }
                            sli.unstack().forEach((t, ind) => {
                                if (ind !== j) {
                                    arr.push(t)
                                } else {
                                    arr.push(newrow)
                                    arr.push(t)
                                }
                            })
                            if (j === r - 1) {
                                arr.push(newrow)
                            }
                            inv = tf.stack(arr)
                        }
                    }
                    return inv
                })
            }
            const trian = tf.unstack(inv)
            len = trian.length
            trian[len - 1] = trian[len - 1].div(trian[len - 1].slice(trian[len - 1].shape[0] - 1, 1).asScalar())
            for (let i = r - 2; i > -1; i--) {
                for (j = r - 1; j > i; j--) {
                    trian[i] = trian[i].sub(trian[j].mul(trian[i].slice(j, 1).asScalar()))
                }
                trian[i] = trian[i].div(trian[i].slice(i, 1).asScalar())
            }
            return tf.split(tf.stack(trian), 2, 1)[1]
        })
    }

    const a = tf.tensor2d([3, 25, 74, 17, 97, 96, 11, 83, 47], [3,3])
    a.print()
    const i = invertM(a)
    i.print()
    callback(i.arraySync())
}
""",callback = callback)

In [143]:
w.debugging_display()